# SED near the Sun's location in the Milky Way

Figure 3.4 from Chapter 3 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot of the spectral energy distribution (SED) of electromagnetic radiation near
the Sun's location in the Milky Way Galaxy. The main peaks are provided
by the cosmic microwave background (CMB), infrared light from dust (IR),
and starlight (SL).

Data are from [Porter et al. 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...846...67P/abstract), using the
model file for x,y,z=(8.5,0,0)kpc, using the [Robataille et al. 2012](https://ui.adsabs.harvard.edu/abs/2012A%26A...545A..39R/abstract) 
(R12) 3D radiative transfer dust and stellar model.

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter, FormatStrFormatter

from astropy.io import fits

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig3_4' 

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## R12 solar location model

The SED is from the Porter et al. data set, using the robitaille_DL07_PAHISMMix_8.5_0_0_Flux.fits data file. 
This file must be in the same folder as this notebook and uncompressed.

Convert the wavelength in microns to energy in eV and use the flux density in eV cm$^{-3}$ as-is.

Also compute the Cosmic Microwave Background (CMB) component and overplot it onto the model SED.

In [ ]:
fitsFile = 'robitaille_DL07_PAHISMMix_8.5_0_0_Flux.fits'

hdu1=fits.open(fitsFile)
data = hdu1[1].data

# Constants

c = 2.99792458e8    # speed of light in m/sec (CODATA 2018)
h = 4.135667696e-15 # eV Hz-1

lam = 1.0e-6*data.field('Wavelength') # wavelength converted from microns to meters

hnuISM = h*c/lam  # energy in eV
fluxISM = data.field('Direct')+data.field('Transient')+data.field('Thermal')

# limits of the ISM data

hnuMin = np.min(hnuISM)
print(f'hnu min = {hnuMin:e}')
fluxMin = np.min(fluxISM)
print(f'flux_min = {fluxMin:e}')
print(f'log10(flux_min) = {math.log10(hnuMin):f}')

xMin = 1.0e-5 # eV
xMax = 14.0

yMin = 1.0e-4 # eV cm^-3
yMax = 5.0

## Cosmic Microwave Background

Compute the flux density as a function of h$\nu$ for the CMB, assuming a temperature of 2.7255K.

Add this into the ISM SED, being mindfull of the different h$\nu$ grids and numerical extremes.  It's a bit
of a hack, but it works OK.

In [ ]:
k_eV = 8.617333262e-5 # Boltzmann's k in eV/K

T = 2.7255 # K

kT = k_eV*T

hnuCMB0 = 10.0**(np.linspace(-5.0,math.log10(hnuMin),101))
fluxCMB0 = 1.318684e13*(hnuCMB0**4)/(np.exp(hnuCMB0/kT)-1)

hnuCMBmax = 10.0**(-1.6)
fluxTot = []
for i,hnu in enumerate(hnuISM):
    if hnu < hnuCMBmax:
        fluxTot.append(fluxISM[i] + 1.318684e13*(hnu**4)/(np.exp(hnu/kT)-1))
    else:
        fluxTot.append(fluxISM[i])


### Make the Plot

Plot energy density vs. photon energy in eV

In [ ]:
fig,ax = plt.subplots()

fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

plt.xlim(xMin,xMax)
ax.set_xscale('log')
plt.xlabel(r'h$\nu$ [eV]')

plt.ylim(yMin,yMax)
ax.set_yscale('log')
plt.ylabel(r'Energy density [eV cm$^{-3}$]')

# plot the SEDs

plt.plot(hnuCMB0,fluxCMB0,'-',color='black',lw=1.0,zorder=10)
plt.plot(hnuISM,fluxTot,'-',color='black',lw=1,zorder=8)

# Label the main parts of the CoG

plt.text(1.0e-3,0.2,'CMB',fontsize=axisFontSize,ha='center',va='bottom')
plt.text(4.0e-2,0.1,'IR',fontsize=axisFontSize,ha='center',va='bottom')
plt.text(2.0,0.6,'Starlight',fontsize=axisFontSize,ha='center',va='bottom')

# plot and file

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')